# Assessing the performance of the model

## Importing

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import root_mean_squared_error as rmse


## Datasets Preparation

In [ ]:
def datasets_preparation(dataset, dataset2, name):
    
    x = np.tile(dataset.x, len(dataset.time_counter)*len(dataset.y))
    y = np.tile(np.repeat(dataset.y, len(dataset.x)), len(dataset.time_counter))
   
    inputs = np.stack([
        np.ravel(dataset2['Summation_of_solar_radiation']),
        np.ravel(dataset2['Mean_wind_speed']),
        np.ravel(dataset2['Mean_air_temperature']),
        y,
        x,
        np.repeat(dataset.time_counter.dt.dayofyear, len(dataset.x)*len(dataset.y)),
        ])

    targets = np.ravel(dataset[name])
    
    indx = np.where(np.isfinite(targets) & (x>10) & ((x>100) | (y<880)))
    inputs = inputs[:,indx[0]]
    targets = targets[indx[0]]

    inputs = inputs.transpose()

    return(inputs, targets, indx)


## Regressor (Training with all)

In [ ]:
def regressor (inputs, targets, table):

    model = TransformedTargetRegressor(regressor=make_pipeline(ColumnTransformer(
        transformers=[('drivers', StandardScaler(), [0,1,2]), ('spatial', KBinsDiscretizer(n_bins=255,encode='ordinal'), [3,4])],remainder='passthrough'),
        HistGradientBoostingRegressor(categorical_features=[3,4,5],learning_rate=0.5)),
        transformer=StandardScaler())
    regr = BaggingRegressor(model, n_estimators=12, n_jobs=4).fit(inputs,targets)
    
    predictions = regr.predict(inputs)
    
    table[0,0] = np.round(np.corrcoef(predictions,targets)[0][1],3)
    table[1,0] = rmse(predictions,targets)
    m,_ = np.polyfit(targets, predictions, deg=1)
    table[2,0] = np.round(m,3)

    return(regr)


## Regressor (Training with 75%, testing with 25%)

In [ ]:
def regressor2 (inputs, targets, table):

    model = TransformedTargetRegressor(regressor=make_pipeline(ColumnTransformer(
        transformers=[('drivers', StandardScaler(), [0,1,2]), ],remainder='passthrough'),
        HistGradientBoostingRegressor(categorical_features=[3,4,5],learning_rate=0.5)),
        transformer=StandardScaler())
    
    X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.25)
    regr = BaggingRegressor(model, n_estimators=12, n_jobs=4).fit(X_train,y_train)

    predictions = regr.predict(X_train)

    table[0,2] = np.round(np.corrcoef(y_train,predictions)[0][1],3)
    table[1,2] = rmse(y_train,predictions)
    m,_ = np.polyfit(y_train,predictions, deg=1)
    table[2,2] = np.round(m,3)
    
    predictions = regr.predict(X_test)

    table[0,3] = np.round(np.corrcoef(y_test,predictions)[0][1],3)
    table[1,3] = rmse(y_test,predictions)
    m,_ = np.polyfit(y_test,predictions, deg=1)
    table[2,3] = np.round(m,3)

    return(regr)


## Evaluation

In [ ]:
def evaluation (ds,ds2,regr,name, table, i):

    dataset = ds.sel(time_counter = slice('2021', '2024'))
    dataset2 = ds2.sel(time_counter = slice('2021', '2024'))

    inputs, targets, _ = datasets_preparation(dataset, dataset2,name)

    predictions = regr.predict(inputs)

    table[0,i] = np.round(np.corrcoef(predictions,targets)[0][1],3)
    table[1,i] = rmse(predictions,targets)
    m,_ = np.polyfit(targets, predictions, deg=1)
    table[2,i] = np.round(m,3)


## Printing

In [ ]:
def printing(table,criteria,categories,metric):

    temp = pd.DataFrame(table.transpose(),columns=criteria,index=categories)
    print(metric)
    display(temp)
    print ('\n')

## Training

In [ ]:
def training(name,table):

    ds = xr.open_dataset('/data/ibougoudis/MOAD/files/integrated_original.nc')
    ds2 = xr.open_dataset('/data/ibougoudis/MOAD/files/external_inputs.nc')

    ds = ds.isel(time_counter = (np.arange(0, len(ds.time_counter),2)), 
        y=(np.arange(ds.y[0], ds.y[-1], 5)), 
        x=(np.arange(ds.x[0], ds.x[-1], 5)))

    ds2 = ds2.isel(time_counter = (np.arange(0, len(ds2.time_counter),2)), 
        y=(np.arange(ds2.y[0], ds2.y[-1], 5)), 
        x=(np.arange(ds2.x[0], ds2.x[-1], 5)))

    dataset = ds.sel(time_counter = slice('2007', '2020'))
    dataset2 = ds2.sel(time_counter = slice('2007', '2020'))

    inputs, targets, _ = datasets_preparation(dataset, dataset2, name)

    regr = regressor(inputs, targets, table)
    evaluation(ds,ds2,regr,name,table,1)

    regr2 = regressor2(inputs, targets, table)
    evaluation(ds,ds2,regr2,name,table,4)


## Evaluation

In [ ]:
criteria = ['r','rms','slope']
categories = ['training with 100%', 'testing', 'training with 75%', 'testing with 25%', 'testing']

diat = np.zeros((len(criteria),len(categories)))
flag = np.zeros((len(criteria),len(categories)))
diat_pr = np.zeros((len(criteria),len(categories)))
flag_pr = np.zeros((len(criteria),len(categories)))

training('Diatom',diat)
training('Flagellate',flag)
training('Diatom_Production_Rate',diat_pr)
training('Flagellate_Production_Rate',flag_pr)


## Printing (Results)

In [31]:
printing(diat,criteria, categories,'Diatom')
printing(flag,criteria, categories,'Flagellate')
printing(diat_pr,criteria, categories,'Diatom production rate')
printing(flag_pr,criteria, categories, 'Flagellate production rate')


Diatom


,r,rms,slope
training with 100%,0.880,0.078760,0.711
testing,0.604,0.133635,0.462
training with 75%,0.877,0.079690,0.704
testing with 25%,0.860,0.084299,0.686
testing,0.603,0.133493,0.458




Flagellate


,r,rms,slope
training with 100%,0.941,0.009364,0.858
testing,0.844,0.015460,0.719
training with 75%,0.941,0.009392,0.856
testing with 25%,0.933,0.009948,0.849
testing,0.845,0.015386,0.722




Diatom production rate


,r,rms,slope
training with 100%,0.942,5.083210e-07,0.866
testing,0.871,7.480932e-07,0.806
training with 75%,0.942,5.099332e-07,0.865
testing with 25%,0.935,5.379212e-07,0.857
testing,0.871,7.473766e-07,0.807




Flagellate production rate


,r,rms,slope
training with 100%,0.950,1.252906e-07,0.884
testing,0.866,2.019755e-07,0.819
training with 75%,0.950,1.260632e-07,0.882
testing with 25%,0.944,1.330290e-07,0.873
testing,0.866,2.018520e-07,0.818
